In [ ]:
compare = "output/Max_temperature" # file name (in the folder "data/model_comparison") of the comparison to be made
consider_models = "all" # list of 4 digit hexa string or "all"
exclude_models = [] # list of 4 digit hexa string or empty list

In [ ]:
# Loading models

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pickle
import json
import pandas as pd
from scipy.signal import savgol_filter
import yaml
from IPython.display import *
import ipywidgets as widgets
from ipywidgets import *
from IPython.display import Image, display

source = f"data/model_comparison/{compare}.yml"

with open(source) as f:
    archives = yaml.safe_load(f)

models = {}

print("Comparing models:")
for model_number, archive in archives.items():

    try:

        details = json.load(open(f'{archive}/details.json'))
        model_id = details["model_id"]

        if consider_models != "all"\
                and model_id not in consider_models\
                or model_id in exclude_models:
            continue

        history = np.load(f'{archive}/history.npz', allow_pickle=True)
        content = np.load(f'{archive}/predictions.npz', allow_pickle=True)
        predictions = content["predictions"].item()
        expectations = content["expectations"].item()

        print(f" - {model_number}: {archive}/details.json")
        print(f"      History keys: {', '.join(list(history.keys()))}")
        
        models[model_id] = {
            "archive": archive,
            "details": details,
            "history": history,
            "predictions": predictions,
            "expectations": expectations
        }
    except:
        print("Could not load model", model_number)



In [ ]:
# Plot training evolution

# Looking at how many different val metrics are available
nb_metrics = 1
seen_metrics = ["loss"]
for model_id, model in models.items():
    for key in model["history"].keys():
        if key.startswith("val_") and not key.endswith("loss") and key not in seen_metrics:
            seen_metrics.append(key)
            nb_metrics += 1

fig, axs = plt.subplots(nb_metrics, 3, figsize=(20, 5*nb_metrics))

for model_id, model in models.items():

    history = model["history"]
    time_range = np.linspace(0, model["details"]["training_time"], model["details"]["epochs"], endpoint=True)

    # Printing loss
    for i, key in enumerate(seen_metrics):
        if key in history:
            for ax in axs[i]:
                curve = history[key]
                ax.plot(time_range, curve, alpha=0.5, label=f"Model {model_id}")


for i, row in enumerate(axs):
    for ax in row:
        ax.set_title(seen_metrics[i])
        ax.set_ylabel(seen_metrics[i])
        ax.set_xlabel('Time (s)')
        ax.legend()
    row[1].set_yscale('log')
    row[2].set_xscale('log')
    row[2].set_yscale('log')

plt.show()

In [ ]:
# Prediction distributions

#==============================================================================
# Violin plots (scalar predictions)
#==============================================================================


def violin_plot(output, model_list, limits):
    plt.figure(figsize=(20, 10))

    labels = []
    def add_label(violin, label):
        color = violin["bodies"][0].get_facecolor().flatten()
        labels.append((mpatches.Patch(color=color), label))

    i = 0
    for key in model_list:
        model = models[key]

        expectations = np.array(model["expectations"][output])  
        predictions = np.array(model["predictions"][output])

        vmin = float(limits[0])
        vmax = float(limits[1])
        eps = float(limits[1]) - float(limits[0]) * 0.01

        expectations = expectations.flatten()
        dim_order = list(range(len(predictions.shape)))
        dim_order.pop(1)
        dim_order.append(1)
        predictions = np.transpose(predictions, dim_order)
        N = predictions.shape[-1]
        S = predictions.size
        predictions = predictions.reshape((S//N,N)).T
        
        mask = ((expectations + eps >= vmin) & (expectations - eps <= vmax)).flatten()
        expectations = expectations[mask]
        predictions = predictions[:,mask]
        

        emin = np.min(expectations)
        emax = np.max(expectations)

        plt.plot([emin,emax],[emin,emax], c="black", linewidth=1)

        violin_parts = plt.violinplot(
            dataset=predictions[:,:],
            positions=expectations[:], 
            widths=(np.max(expectations)-np.min(expectations))*0.05, showmeans=True, showextrema=False
        )

        add_label(violin_parts, f"Model {key}")
 
    plt.title(f'Model predictions on {output}')
    plt.ylabel('Predictions')
    plt.xlabel('Expected')
    plt.ticklabel_format(axis="both", style='sci', scilimits=(0,0))
    plt.grid()
    plt.legend(*zip(*labels), loc=2)
    plt.show()

#==============================================================================
# GUI
#==============================================================================

if compare.startswith("output/"):
    outputs = [compare.split("/")[1]]
elif compare.startswith("problem/"):
    outputs = compare.split("/")[1].split("---")[1].split(",")
else:
    raise ValueError("Invalid compare argument")

model_list_select = widgets.SelectMultiple(
    options=list(models.keys()),
    value=list(models.keys()),
    description='Models',
    disabled=False
)

sample_model = models[list(models.keys())[0]]
output_widget = Dropdown(options=outputs, value=outputs[0], description="Output: ")

def get_possible_expectations():
    possible_expectations = np.array([])
    for model in models.values():
        possible_expectations = np.concatenate((
            possible_expectations,
            model["expectations"][output_widget.value].flatten()
        ))

    possible_expectations = list(map(lambda x: f"{x:.2e}", np.sort(possible_expectations).tolist()))
    return possible_expectations

expectation_slider = widgets.SelectionRangeSlider(
    options=get_possible_expectations(),
    index=(0, len(get_possible_expectations())-1),
    description='Range',
    disabled=False,
    width='100%',
    layout=Layout(width='100%')  
)

plot_button = widgets.Button(
    description='Plot',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

def update_violin_plot(*args):
    clear_output()
    display(model_list_select)
    if len(outputs) > 1:
        display(output_widget)
    display(expectation_slider)
    display(plot_button)

    violin_plot(output_widget.value, model_list_select.value, expectation_slider.value)

plot_button.on_click(update_violin_plot)
update_violin_plot()

In [ ]:
# Select model and view details

def print_model_details(id):

    model = models[id]
    archive = model["archive"]
    details = model["details"]

    display(Image(filename=f'{archive}/model.png'))
    print("\n".join(details['summary'][-4:]))

    print("")
    print("Optimizer:", details['optimizer'])
    if isinstance(details['loss'], list):
        print("Loss:\n -", "\n - ".join(details['loss']))
    else:
        print("Loss:", details['loss'])
    print("Metrics:\n -", "\n - ".join(details['metrics']))
    print("")
    print("Epochs:", details['epochs'])
    print("Batch size:", details['batch_size'])
    print("")
    print(f"Training time: {int(details['training_time']//60)}m {details['training_time']%60:.2f}s" )
    print("Score:\n -", "\n - ".join([f"{i:.2e}" for i in details['scores']]))
    print("")
    # print("Dataset size:", details['dataset_size'])
    print("Validation fraction:", int(details['val_frac'] * 100), "%")
    print("Test fraction:", int(details['test_frac'] * 100), "%")
    print("")
    print("Path:", archive)
    print("")
    if "inputs" in details:
        print("Inputs:\n -", "\n - ".join(details['inputs']))
    if "outputs" in details:
        print("Outputs:\n -", "\n - ".join(details['outputs']))

options = []
for model in models.values():
    options.append(model["details"]["model_id"])

selected_model = options[0]

model_widget = Dropdown(options=options, value=options[0], description="Model ID: ")

# Define a function that updates the content of y based on what we select for x
def update_model_widget(*args):
    global selected_model
    clear_output()
    display(model_widget)
    print_model_details(model_widget.value)
    selected_model = models[model_widget.value]

model_widget.observe(update_model_widget)
update_model_widget()